<a href="https://colab.research.google.com/github/akitim/SkillFactory_current/blob/main/Unit_3_ML_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PYTHON-11. Feature Engineering

In [ ]:
import pandas as pd
#import matplotlib
import matplotlib.pyplot as plt
# import seaborn as sns
import warnings; warnings.filterwarnings('ignore') #(action='once')
import missingno as msno
# from itertools import combinations
# from scipy.stats import ttest_ind
# from scipy.stats import norm
 
%matplotlib inline
 
pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок
pd.options.display.precision = 3
 
 
#import sys
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#sys.path.clear
#sys.path.append('/content/gdrive/MyDrive/SkillFactory')

#Загрузка

In [ ]:
log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']
#log = pd.read_csv('/content/gdrive/MyDrive/SkillFactory/log.csv', names=cols)
log = pd.read_csv(log_path, names=cols)
 
#users = pd.read_csv(
#    '/content/gdrive/MyDrive/SkillFactory/vsers.csv',
#    encoding='koi8-r',
#    sep = '\t'
#)
 
users_path = 'https://raw.githubusercontent.com/akitim/datasets/main/users.csv'
users = pd.read_csv(
    users_path,
    encoding='koi8-r',
    sep = '\t'
)
users.columns = ['user_id', 'email', 'geo']
users.sample(1)
# log.sample(1)

,Юзверь,мейл,Гео
52,User_904,Kryptoburs@yahoo.com,Ярославль


Задание 1-5

In [ ]:
len(log.loc[log.loc[:, 'time'].isna(), 'time'])

15

In [ ]:
#log.dropna(axis=1)
 
#log.dropna(axis=0)
 
#log.loc[users.index.intersection(['time', 'user_id'])].dropna(axis=1)
for col in ['time', 'user_id']:
    if log.loc[:, col].isna().any():
         log.drop(col, inplace=True, axis=1)
log

,user_id,bet,win
0,Запись пользователя № - user_919,NaN,NaN
1,Запись пользователя № - user_973,NaN,NaN
2,Запись пользователя № - user_903,NaN,NaN
3,Запись пользователя № - user_954,NaN,NaN
4,Запись пользователя № - user_954,NaN,NaN
...,...,...,...
995,Запись пользователя № - user_984,9754.0,NaN
996,#error,10054.0,29265.0
997,#error,10454.0,NaN
998,#error,1000.0,NaN


In [ ]:
len(log[['time','user_id']].drop_duplicates())

986

11.3 Преобразование к datetime

In [ ]:
log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']
log = pd.read_csv(log_path, names=cols)

log = log.dropna()

time = log.loc[:, 'time']
time = pd.to_datetime(
    time.apply(lambda t: t[1:] if type(t) == str else t),
    
)
time.apply(lambda t: t.minute)

14     57
29     59
151    54
189    34
205    26
       ..
967    25
971    35
972    35
976    35
991    55
Name: time, Length: 133, dtype: int64

#11.4 Извлечение признаков времени

In [ ]:
#Задание 1

log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']

log = pd.read_csv(log_path, names=cols)
log.dropna(inplace=True)

log.loc[:, 'time'] = pd.to_datetime(
    log.loc[:, 'time'].apply(lambda t: t[1:] if type(t) == str else t),
)
log.loc[:, 'time'].dt.minute.value_counts().index[0]
# pd.DataFrame(log.loc[:, 'time'].dt.minute.value_counts())
# pd.DataFrame(log.loc[:, 'time'].dt.second).hist();
# pd.DataFrame(log.loc[:, 'time'].dt.minute).hist();
# pd.DataFrame(log.loc[:, 'time'].dt.hour).hist();

# Задание 2
log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']

log = pd.read_csv(log_path, names=cols)
log.dropna(inplace=True)
log.loc[:, 'time'] = pd.to_datetime(
    log.loc[:, 'time'].apply(lambda t: t[1:] if type(t) == str else t)
)
log.loc[:, 'time'].dt.month.value_counts() #.index[0]

3    57
4    51
2    16
1     9
Name: time, dtype: int64

In [ ]:
# Задание 3*
log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']

log = pd.read_csv(log_path, names=cols)
log.dropna(inplace=True)

log.loc[:, 'time'] = pd.to_datetime(
    log.loc[:, 'time'].apply(lambda t: t[1:] if type(t) == str else t),
)

weekend = log.loc[:, 'time'].dt.weekday.apply (lambda d: d in [5, 6])
len(log.loc[weekend, 'time'])

log[(log.time.dt.weekday == 5) | (log.time.dt.weekday == 6)]['time'].count()

37

# Задание 4*

In [ ]:
log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']

log = pd.read_csv(log_path, names=cols)
# log.dropna(inplace=True)

log.loc[:, 'time'] = pd.to_datetime(
    log.loc[:, 'time'].apply(lambda t: t[1:] if type(t) == str else t),
)

def timeofday(h):
    if h >= 0 and h <= 5:
        return 'night'
    elif h >=6 and h <= 11:
        return 'morning'
    elif h >= 12 and h <= 17:
        return 'day'
    else:
        return 'evening'

time = log.loc[:, 'time'].dropna()
drare = time.dt.hour.apply(timeofday).value_counts().min()
print(
time.dt.hour.apply(timeofday).value_counts() [
    time.dt.hour.apply(timeofday).value_counts() == drare
].index[0]
)

evening


#11.5 Снова смотрим на пропуски

In [ ]:
log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']

log = pd.read_csv(log_path, names=cols)

log.loc[:, 'bet'].fillna(0, inplace=True)
log.loc[:, 'bet'].value_counts()[0]

515

In [ ]:
log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']
 
log = pd.read_csv(log_path, names=cols)
log.loc[:, 'bet'].fillna(0, inplace=True)
 
def fillna_win(row):
    if pd.isna(row['win']) and pd.isna(row['bet']):
        row['win'] = 0
        return row
    elif row['bet'] == 0:
        row['win'] = 0
        return row
    elif pd.isna(row['win']) and not(pd.isna(row['bet'])):
        row['win'] = -row['bet']
        return row
    else:
        return row
log = log.apply(lambda row: fillna_win(row), axis=1)
(log.loc[:, 'win'] < 0).sum()
 
#11.6 Есть ли смысл играть?
#Задание 1
log.loc[:, 'net'] = -log.loc[:, 'bet']
log.loc[log.loc[:, 'win'] >= 0, 'net'] = log.loc[:,'win'] - log.loc[:, 'bet']
(log.loc[:, 'net'] > 0).sum()
 
#Задание 2
round(
    log.loc[log.loc[:, 'net'] > 0, 'net'].mean()
)   # => 80253
 
#Задание 3
round(
    log.loc[log.loc[:, 'net'] > 0, 'net'].median()
)   # => 5347 

# log.loc[log.loc[:, 'net'] < 1e7, 'net'].hist(bins=50, log=True);
# plt.xlabel('net');
# plt.title('Net distribution');

80253

#11.7 Сколько можно выиграть/проиграть?

In [ ]:
import pandas as pd

log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']
log = pd.read_csv(log_path, names=cols)

log.loc[:, 'bet'].fillna(0, inplace=True)
log.loc[:, 'win'].fillna(0, inplace=True)

log.loc[:, 'net'] = log.loc[:, 'win'] - log.loc[:, 'bet']

#log.loc[log.loc[:, 'bet'] > 0, 'bet'].count()

#Задание 1
round(
    (log.loc[:, 'bet'] > 0).sum() / len(log) * 100,
    #(log.loc[:, 'bet'] > 0).mean() * 100
    1
)

#Задание 2
log.loc[log.loc[:, 'bet'] > 0, 'bet'].mean() // 1

#Задание 3
log.loc[log.loc[:, 'bet'] > 0, 'net'].mean() // 1

#Задание 4
round(log.loc[log.loc[:, 'net'] < 0, 'net'].mean()) + 1

#Задание 5
#процент выигрышных ставок
winbet = ((log.loc[:, 'bet'] > 0) * (log.loc[:, 'net'] > 0)).mean() * 100
winbet < 100 - winbet

True

In [ ]:
#Задание 6*
import pandas as pd

def r2i(f):
    return round(f - 0.5)

log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']
log = pd.read_csv(log_path, names=cols)

log.loc[:, 'bet'].fillna(0, inplace=True)
log.loc[:, 'win'].fillna(0, inplace=True)

min_bet = log.loc[log.loc[:, 'bet'] > 0, 'bet'].min()
min_bet_amount = r2i(
    log.loc[log.loc[:, 'bet'] == min_bet, 'bet'].count()
)


#11.8 Повторение merge/groupby

In [4]:
import pandas as pd

def r2i(f):
    return round(f - 0.5)

log_path = 'https://raw.githubusercontent.com/akitim/datasets/main/log.csv'
cols = ['user_id', 'time', 'bet', 'win']
log = pd.read_csv(log_path, names=cols)

us_path = 'https://raw.githubusercontent.com/akitim/datasets/main/users.csv'
us = pd.read_csv(
    us_path,
    encoding='koi8-r',
    sep = '\t'
)
us.columns = ['user_id', 'email', 'geo']

# Приведем признак user_id к одному формату в обоих датасетах  
us.user_id = us.user_id.apply(lambda x: x.lower())

# Избавимся от ошибок в user_id  
log = log[log.user_id != '#error']  
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])

log.loc[:, 'bet'].fillna(0, inplace=True)
log.loc[:, 'win'].fillna(0, inplace=True)

log.loc[:, 'net'] = log.loc[:, 'win'] - log.loc[:, 'bet']

logus = pd.merge(log, us, on='user_id')

logus.groupby('user_id')['win'].median().median()  

#11.9 Анализ по пользователям
# Задание 1
r2i(logus.groupby(by='user_id')['net'].sum().median())

#Задание 2*
# logus.groupby(by='user_id')['bet'].sum().sort_values(ascending=True)

unix = logus.loc[logus.loc[:, 'bet'] > 0, 'user_id'].unique()

r2i(
    logus.loc[
    logus.loc[:, 'bet'] == 0,
    'user_id'
    ].apply(lambda u: u if u in unix else '').value_counts().mean()
) # => 5

#Задание 3**
# Сколько в среднем времени проходит
# между появлением человека в сервисе и первой ставкой?

logus.loc[:, 'time'] = pd.to_datetime(
    logus.loc[:, 'time'].apply(lambda t: t[1:])
)

def time2bet (id):
    tb = logus.loc[
        logus.loc[:, 'user_id'] == id, 'time':'bet'
    ].sort_values(by='time')

    t_appear = tb.loc[tb.index.min(), 'time']
    t_1stbet = tb.loc[tb.loc[tb.loc[:, 'bet'] > 0, 'time'].index.min(), 'time']

    return (t_1stbet - t_appear).total_seconds() / 3600 / 24

unix = logus.loc[logus.loc[:, 'bet'] > 0, 'user_id'].unique()

dt = pd.DataFrame(data=[time2bet(u) for u in unix], index = unix, columns=['time_to_1stbet'])
r2i(dt.loc[:, 'time_to_1stbet'].mean())
# time2bet('user_902')

47

#11.10 Анализ по городам

In [16]:
# Задание 1
logus.groupby(by='geo')['win'].sum().sort_values(ascending=False).index[1]

# Задание 2*
mean_bet_by_city = logus.groupby(by='geo')['bet'].mean() #.sort_values(ascending=False)
r2i(mean_bet_by_city.max() / mean_bet_by_city.min())
mean_bet_by_city

geo
Арзангелтск          663.708333
Воронеж             1324.840909
Екатеринбург         228.571429
Ижевск              1079.836066
Казань               714.575758
Краснодар            839.813953
Красноярск          1428.178571
Москва             39793.327869
Пермь               1135.818182
Санкт-Петербург      654.521739
Ставрополь          1582.500000
Тюмень               175.000000
Хабаровск            757.700000
Ярославль            750.000000
Name: bet, dtype: float64